如下我们要验证 vLLM 的 continuous batching 过程，以及背后的一些机理和有趣的实验现象。

请先运行 vllm_server.py 启动 vllm 服务，将如下脚本粘贴到外部 .py 文件中运行。

In [ ]:
from vllm import LLM, SamplingParams
from vllm.entrypoints.openai.api_server import run_server
import asyncio

MODEL_PATH = "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-mtsearch-assistant/ai-search/deepsearch_files/LLMbasemodels/huggingface.co/Qwen/Qwen3-30B-A3B-Instruct-2507"

# vLLM engine configuration
extra_kwargs = {
    "tensor_parallel_size": 1,
    "gpu_memory_utilization": 0.75,
    "max_model_len": 4096,
}

# Default generation parameters
generation_kwargs = {
    "temperature": 0.3,
    "max_tokens": 512,
    "top_p": 0.9,
}

if __name__ == "__main__":
    import subprocess
    import sys
    
    # Start vLLM OpenAI-compatible API server
    cmd = [
        "vllm", "serve",
        MODEL_PATH,
        "--tensor-parallel-size", str(extra_kwargs["tensor_parallel_size"]),
        "--gpu-memory-utilization", str(extra_kwargs["gpu_memory_utilization"]),
        "--max-model-len", str(extra_kwargs["max_model_len"]),
        "--host", "0.0.0.0",
        "--port", "8000",
    ]
    
    print(f"Starting vLLM server with command:")
    print(" ".join(cmd))
    print(f"\nServer will be available at: http://0.0.0.0:8000")
    print(f"API documentation: http://0.0.0.0:8000/docs")
    
    subprocess.run(cmd)

In [4]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="",
)

response = client.chat.completions.create(
    model="/mnt/dolphinfs/hdd_pool/docker/user/hadoop-mtsearch-assistant/ai-search/deepsearch_files/LLMbasemodels/huggingface.co/Qwen/Qwen3-30B-A3B-Instruct-2507",
    messages=[{"role": "user", "content": "Hello, how are you?"}],
    temperature=0.3,
    max_tokens=512,
    top_p=0.9,
)

response.choices[0].message.content

"Hello! I'm functioning well, thank you for asking! 😊 I'm here and ready to help you with whatever you need. How can I assist you today?"

接下来我们验证 vLLM 的 continuous batching。

In [8]:
# Test prompts with different lengths
test_prompts = [
    "今天天气怎么样？",
    "请用三句话介绍一下你最喜欢的一本书。",
    "写一首关于秋天的七言绝句。",
    "解释量子计算和经典计算的区别，并举例说明。",
    "推荐三个适合周末游玩的城市景点。",
    "什么是区块链技术？请简要说明其工作原理。",
    "用 Python 实现一个计算斐波那契数列的函数。",
    "描述一下你理想中的假期是什么样的。",
    "如何制作一杯美味的咖啡？",
    "请解释一下相对论的基本概念。",
]

In [9]:
from openai import AsyncOpenAI

client = AsyncOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="",
)

首先写一个并发发送多个请求的 demo，使得 vLLM 在后台确实实现了动态批处理。基于 asyncio 实现并发任务 gather。

In [18]:
import asyncio
import time

async def generate_responses(prompt) -> tuple[str, float]:
    start_time = time.time()
    try:
        response = await client.chat.completions.create(
            model="/mnt/dolphinfs/hdd_pool/docker/user/hadoop-mtsearch-assistant/ai-search/deepsearch_files/LLMbasemodels/huggingface.co/Qwen/Qwen3-30B-A3B-Instruct-2507",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=512,
            top_p=0.9,
        )
        length = len(response.choices[0].message.content)
        end_time = time.time()
        latency = end_time - start_time
        return response.choices[0].message.content, latency, length
    except Exception as e:
        return None, 0

In [19]:
async def continuous_batching():
    tasks = [generate_responses(prompt) for prompt in test_prompts]
    start_time = time.time()
    results = await asyncio.gather(*tasks)
    end_time = time.time()
    latency_all = end_time - start_time
    return results, latency_all

async def main():
    results, latency_all = await continuous_batching()
    print(f"Total latency: {latency_all} seconds")
    is_bigger = True
    for prompt, (response, latency, length) in zip(test_prompts, results):
        print(f"Prompt: {prompt}\nResponse: {repr(response[:50])}\nLength: {length}\nLatency: {latency} seconds\n")
        if latency > latency_all:
            is_bigger = False
    print(f"Is latency_all the biggest? {is_bigger}")

await main()

Total latency: 9.954436302185059 seconds
Prompt: 今天天气怎么样？
Response: '我无法获取实时天气信息。建议你查看当地的天气预报，比如通过天气App、搜索引擎搜索“你所在城市+天气'
Length: 89
Latency: 1.3015964031219482 seconds

Prompt: 请用三句话介绍一下你最喜欢的一本书。
Response: '我最喜欢的一本书是《小王子》。它以简洁而诗意的语言，讲述了一个来自外星的小王子探索宇宙、寻找爱与意义'
Length: 97
Latency: 1.4555788040161133 seconds

Prompt: 写一首关于秋天的七言绝句。
Response: '金风拂叶舞斜阳，  \n霜染层林映晚光。  \n雁阵横天书字去，  \n寒江独钓一蓑霜。'
Length: 41
Latency: 0.871971845626831 seconds

Prompt: 解释量子计算和经典计算的区别，并举例说明。
Response: '量子计算与经典计算是两种截然不同的计算范式，它们在基本原理、信息表示方式和处理能力上存在根本区别。以'
Length: 923
Latency: 9.950427293777466 seconds

Prompt: 推荐三个适合周末游玩的城市景点。
Response: '当然可以！以下是三个适合周末游玩的城市景点推荐，涵盖自然风光、人文历史与休闲体验，适合不同兴趣的游客'
Length: 743
Latency: 9.344392776489258 seconds

Prompt: 什么是区块链技术？请简要说明其工作原理。
Response: '区块链技术是一种分布式账本技术，它通过去中心化、加密和共识机制，确保数据的安全、透明和不可篡改。\n\n'
Length: 525
Latency: 6.176791429519653 seconds

Prompt: 用 Python 实现一个计算斐波那契数列的函数。
Response: '以下是几种用 Python 实现斐波那契数列的函数，从简单到高效：\n\n## 方法1：递归实现（简单但'
Length: 1216
Latency: 9.948890924453735 seconds



然后写一个 sequential 的串行场景，模仿无并发的情况。

In [20]:
start_time = time.time()
for prompt in test_prompts:
    response, latency, length = await generate_responses(prompt)
    print(f"Prompt: {prompt}\nResponse: {repr(response[:50])}\nLength: {length}\nLatency: {latency} seconds\n")
end_time = time.time()
latency_all = end_time - start_time
print(f"Total latency: {latency_all} seconds")

Prompt: 今天天气怎么样？
Response: '我无法获取实时天气信息。建议您查看当地的天气预报应用或网站，如中国气象局官网、天气通、墨迹天气等，以'
Length: 60
Latency: 0.35826635360717773 seconds

Prompt: 请用三句话介绍一下你最喜欢的一本书。
Response: '我最喜欢的一本书是《小王子》。它以简洁而诗意的语言，讲述了一位来自外星的小王子在宇宙中旅行、探索爱与'
Length: 97
Latency: 0.537745475769043 seconds

Prompt: 写一首关于秋天的七言绝句。
Response: '金风拂叶舞斜阳，  \n霜染层林映晚光。  \n雁字横天书秋意，  \n一江寒水送归航。'
Length: 41
Latency: 0.300952672958374 seconds

Prompt: 解释量子计算和经典计算的区别，并举例说明。
Response: '量子计算与经典计算是两种截然不同的信息处理范式，它们在基本原理、计算方式和应用潜力上存在根本区别。以'
Length: 854
Latency: 4.184512376785278 seconds

Prompt: 推荐三个适合周末游玩的城市景点。
Response: '当然可以！以下是三个适合周末游玩的城市景点推荐，涵盖自然风光、人文历史与休闲娱乐，适合不同兴趣的游客'
Length: 762
Latency: 4.192795753479004 seconds

Prompt: 什么是区块链技术？请简要说明其工作原理。
Response: '区块链技术是一种分布式账本技术，它通过去中心化、加密和共识机制，确保数据的安全、透明和不可篡改。\n\n'
Length: 507
Latency: 2.5272810459136963 seconds

Prompt: 用 Python 实现一个计算斐波那契数列的函数。
Response: '以下是几种用 Python 实现计算斐波那契数列的函数，从简单到高效：\n\n## 方法1：递归实现（简'
Length: 1208
Latency: 4.185424089431763 seconds

Prompt: 描述一下你理想中的假期是什么样的。
Response: '我理想

如上，我们确实可以看到不同的 prompts 完成的时间各自不同，有的快有的慢，总体时间是 9.8 秒左右；不过，我们现在要比较的是它和（1）完全不 batching，即顺序 sequential 发送和处理（2）普通 batching，拿一批样本，padding 到同一长度后一次 forward 结束，弊端是短请求被长请求拖死的情况，类似基于 transformers 实现。

我们需要观测到**确实**发生了 continuous batching，最有力的证据应该是不同 query 的处理顺序标记；而现在的情况也有可能是 vLLM 底层也只是退化地完成了普通 batching，毕竟当前的 batch 大小只有 10。根据如上结果可以看到，sequential 用了 28 秒左右，接近所有 test query 推理的时间总和。然而，我们也发现，相较于 asyncio 并发的情况，sequential 每条数据的推理时间几乎减少了一倍之多！这背后的原因是什么？目前的猜测大致是，由于串行使得 QPS 都用于当前 query 的推理，故而加速了？但具体的机制尚不清楚，毕竟如果是 batch 并行的，应该是不会牺牲单个 query 的速度的？

让我们继续探究。